In [1]:
def request_flags(method, url):
    if ssl_flag:
        return request(method, url)
    else:
        return request(method, url, verify=False)

### get info using bs id

In [2]:
def query_dataservice_bs_id(url, bs_id, bs_attrs, pt_attrs, dx_attrs):
    p = 5
    bs_id = bs_id.rstrip('\n')
    bs_url = url + '/biospecimens/' + bs_id
    # sys.stderr.write(bs_url + '\n')
    bs_info = ''
    try:
        bs_info = request_flags('GET', bs_url)
    except Exception as e:
        sys.stderr.write('Got error ' + str(e) + ', trying again after a pause of ' + str(p) + ' seconds\n')
        sleep(5)
        try:
            bs_info = request_flags('GET', bs_url)
        except Exception as e1:
            sys.stderr.write('Repeat failed with error ' + str(e1) + '\n')
            exit(1)
    result = []
    if bs_info.json()['_status']['code'] == 404:
        result.append(bs_info.json()['_status']['message'])
        sys.stderr.write(bs_id + ' not found!\n')
        return bs_id, result
    dx_url = url + bs_info.json()['_links']['diagnoses']
    dx_dict = {}
    dx_obj = ''
    try:
        dx_obj = request_flags('GET', dx_url) if len(dx_attrs) > 0 else 'NoDX'
    except Exception as e:
        sys.stderr.write('Got error ' + str(e) + ', trying again after a pause of ' + str(p) + ' seconds\n')
        sleep(5)
        try:
            dx_obj = request_flags('GET', dx_url) if len(dx_attrs) > 0 else 'NoDX'
        except Exception as e1:
            sys.stderr.write('Repeat failed with error ' + str(e1) + '\n')
            exit(1)
    # dir(bs_info)
    pt_url = bs_info.json()['_links']['participant']
    pt_info = ''
    if len(pt_attrs) > 0:
        try:
            pt_info = request_flags('GET', url + pt_url)
    
        except Exception as e:
            sys.stderr.write('Got error ' + str(e) + ', trying again after a pause of ' + str(p) + ' seconds\n')
            sleep(5)
            try:
                pt_info = request_flags('GET', url + pt_url)
            except Exception as e1:
                sys.stderr.write('Repeat failed with error ' + str(e1) + '\n')
                exit(1)
        result.append(pt_info.json()['results']['kf_id'])
    for attr in bs_attrs:
        # sys.stderr.write(attr + ': ')
        res = bs_info.json()['results'][attr]
        if res is None:
            res = 'NULL'
        # sys.stderr.write(res + '\n')
        result.append(str(res))
    for attr in pt_attrs:
        # sys.stderr.write(attr + ': ')
        res = pt_info.json()['results'][attr]
        if res is None:
            res = 'NULL'
        # sys.stderr.write(res + '\n')
        result.append(res)
    for attr in dx_attrs:
        dx_dict[attr] = []
        for cur_res in dx_obj.json()['results']:
            dx_dict[attr].append(str(cur_res[attr]))
        result.append(';'.join(dx_dict[attr]))
                                 
    return bs_id, result
    

### Query dataservice by pt ID

In [4]:
def query_dataservice_pt_id(url, pt_id, bs_attrs, pt_attrs, dx_attrs, out_fh):
    pt_id = pt_id.rstrip('\n')
    pt_url = url + '/participants/' + pt_id
    try:
    # sys.stderr.write(bs_url + '\n')
        pt_info = request_flags('GET', pt_url)
        bs_url = pt_info.json()['_links']['biospecimens']
        bs_list = request_flags('GET', url + bs_url)
        res_set = []
        for bs_info in bs_list.json()['results']:
            dx_url = url + bs_info['_links']['diagnoses']
            dx_dict = {}
            dx_obj = request_flags('GET', dx_url) if len(dx_attrs) > 0 else 'NoDX'
            # dir(bs_info)

            result = []
            result.append(bs_info['kf_id'])
            result.append(pt_info.json()['results']['kf_id'])

            for attr in bs_attrs:
                # sys.stderr.write(attr + ': ')
                res = bs_info[attr]
                if res is None:
                    res = 'NULL'
                # sys.stderr.write(res + '\n')
                result.append(res)
            for attr in pt_attrs:
                # sys.stderr.write(attr + ': ')
                res = pt_info.json()['results'][attr]
                if res is None:
                    res = 'NULL'
                # sys.stderr.write(res + '\n')
                result.append(res)
            for attr in dx_attrs:
                dx_dict[attr] = []
                for cur_res in dx_obj.json()['results']:
                    dx_dict[attr].append(str(cur_res[attr]))
                result.append(';'.join(dx_dict[attr]))
            res_set.append('\t'.join(result))
        return res_set
    except Exception as e:
        sys.stderr.write(str(e) + '\n')
        sys.stderr.write('Had trouble processing ' + pt_id + '\n')

## set up imports and output file

In [5]:
import sys
from requests import request
import concurrent.futures
from time import sleep
# set to 1 for normal operation, 0 to ignore cert validation
ssl_flag = 1
fname = '/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/peddy_results/bs_id_list.txt'
file_list = fname
bs_attrs = ['external_aliquot_id', 'analyte_type', 'source_text_tissue_type', 'composition', 'external_sample_id']
# pt_attrs = ['external_id', 'gender', 'ethnicity', 'race']
pt_attrs = []
# dx_attrs = ['source_text_diagnosis', 'age_at_event_days', 'source_text_tumor_location']
dx_attrs = []
url = 'https://kf-api-dataservice.kidsfirstdrc.org/'
out_fh = open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/peddy_results/tum_ds_info.txt', 'w')
out_fh.write('BS_ID\tPT_ID\t' + '\t'.join(bs_attrs) + '\t' + '\t'.join(pt_attrs))
if len(dx_attrs) > 0:
    out_fh.write('\t' + '\t'.join(dx_attrs) )
out_fh.write('\n')

    

1

## get by bs id

In [6]:

x = 1
m = 10
th = 24
with concurrent.futures.ThreadPoolExecutor(th) as bs_exec:
    bs_results = {
    bs_exec.submit(query_dataservice_bs_id, url, bs_id, bs_attrs, pt_attrs, dx_attrs): bs_id for bs_id in open(file_list)}
    for bs_result in concurrent.futures.as_completed(bs_results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' bs ids\n')
        (cur_bs_id, bs_info) = bs_result.result()
        out_fh.write(cur_bs_id + '\t' + '\t'.join(bs_info) + '\n')
        x += 1
out_fh.close()
sys.stderr.write('Done!')

Processed 10 bs ids
Processed 20 bs ids
Processed 30 bs ids
Processed 40 bs ids
Processed 50 bs ids
Processed 60 bs ids
Processed 70 bs ids
Processed 80 bs ids
Processed 90 bs ids
Processed 100 bs ids
Processed 110 bs ids
Processed 120 bs ids
Processed 130 bs ids
Processed 140 bs ids
Processed 150 bs ids
Processed 160 bs ids
Processed 170 bs ids
Processed 180 bs ids
Processed 190 bs ids
Processed 200 bs ids
Processed 210 bs ids
Processed 220 bs ids
Processed 230 bs ids
Processed 240 bs ids
Processed 250 bs ids
Processed 260 bs ids
Processed 270 bs ids
Processed 280 bs ids
Processed 290 bs ids
Processed 300 bs ids
Processed 310 bs ids
Processed 320 bs ids
Processed 330 bs ids
Processed 340 bs ids
Processed 350 bs ids
Processed 360 bs ids
Processed 370 bs ids
Processed 380 bs ids
Processed 390 bs ids
Processed 400 bs ids
Processed 410 bs ids
Processed 420 bs ids
Processed 430 bs ids
Processed 440 bs ids
Processed 450 bs ids
Processed 460 bs ids
Processed 470 bs ids
Processed 480 bs ids
P

## Get by pt id

In [9]:
#out_fh = open('/Users/brownm28/Documents/2019-Mar-22_cbttc-hakon/hakon_metadata.txt', 'w')
x = 1
m = 25
th = 40
with concurrent.futures.ThreadPoolExecutor(th) as pt_exec:
    pt_results = {
    pt_exec.submit(query_dataservice_pt_id, url, pt_id, bs_attrs, pt_attrs, dx_attrs, out_fh): pt_id for pt_id in open(file_list)}
    for pt_result in concurrent.futures.as_completed(pt_results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' pt ids\n')
        (res_list) = pt_result.result()
        out_fh.write('\n'.join(res_list) + '\n')
        x += 1
out_fh.close()
sys.stderr.write('Done!')


Processed 25 pt ids
Processed 50 pt ids
Processed 75 pt ids
Processed 100 pt ids
Processed 125 pt ids
Processed 150 pt ids
Processed 175 pt ids
Processed 200 pt ids
Processed 225 pt ids
Processed 250 pt ids
Processed 275 pt ids
Processed 300 pt ids
Processed 325 pt ids
Processed 350 pt ids
Processed 375 pt ids
Processed 400 pt ids
Processed 425 pt ids
Processed 450 pt ids
Processed 475 pt ids
Processed 500 pt ids
Processed 525 pt ids
Processed 550 pt ids
Processed 575 pt ids
Processed 600 pt ids
Processed 625 pt ids
Processed 650 pt ids
Processed 675 pt ids
Processed 700 pt ids
Processed 725 pt ids
Processed 750 pt ids
Processed 775 pt ids
Processed 800 pt ids
Processed 825 pt ids
Processed 850 pt ids
Processed 875 pt ids
Processed 900 pt ids
Processed 925 pt ids
Processed 950 pt ids
Processed 975 pt ids
Processed 1000 pt ids
Processed 1025 pt ids
Processed 1050 pt ids
Processed 1075 pt ids
Processed 1100 pt ids
Processed 1125 pt ids
Processed 1150 pt ids
Processed 1175 pt ids
Processe

## get info using cram - bs id pairs

In [ ]:
for cram_id_pair in open(file_list):
    (cram, bs_id) = cram_id_pair.rstrip('\n').split('\t')
    sys.stderr.write(cram + '\t' + bs_id + '\n')
    bs_info = query_dataservice(url, bs_id, bs_attrs, pt_attrs)
    sys.stdout.write(bs_id + '\t' + '\t'.join(bs_info) + '\t' + cram + '\n')

### Get PT from study

In [5]:
def process_pt_batch(cur_pt_res_dict, pt_ids):
    pt_id = cur_pt_res_dict['kf_id']
    pt_ids.append(pt_id)


In [6]:
def luke_study_walker(url, study_id):
    f = 0
    lim = 100
    x = lim
    next_link = ''
    init_link = url + '/participants?study_id=' + study_id + '&limit=' + str(lim)
    sys.stderr.write('Processing first batch of ' + str(lim) + ' ' + init_link + '\n')
    init_pt = request('GET', init_link)
    pt_ids = []

    if 'next' in init_pt.json()['_links']:
        next_link = url + init_pt.json()['_links']['next'] + '&limit=' + str(lim)
    else:
        f = 1
    # process_pt_batch(url, init_pt.json()['results'], file_dict, pt_bs_id_dict)
    with concurrent.futures.ThreadPoolExecutor(40) as pt_exec:
        pt_results = {pt_exec.submit(process_pt_batch, pt_res_dict, pt_ids): pt_res_dict
                      for pt_res_dict in init_pt.json()['results']}
    while f == 0:
        x += lim
        sys.stderr.write('Processing next batch of ' + str(lim) + ' ' + next_link + '\n')
        sys.stderr.flush()
        next_pt = request('GET', next_link)
        if 'next' in next_pt.json()['_links']:
            next_link = url + next_pt.json()['_links']['next'] + '&limit=' + str(lim)
        else:
            f = 1
            sys.stderr.write('Last batch\n')
        # multi-thread iterating through patient links
        with concurrent.futures.ThreadPoolExecutor(40) as pt_exec:
            pt_results = {pt_exec.submit(process_pt_batch, pt_res_dict, pt_ids): pt_res_dict
                      for pt_res_dict in next_pt.json()['results']}

    return pt_ids

In [7]:
import sys
from requests import request
import concurrent.futures
from time import sleep

study_id = 'SD_JWS3V24D'
url = 'https://kf-api-dataservice.kidsfirstdrc.org/'
pt_ids = luke_study_walker(url, study_id)
out = open('/Users/brownm28/Documents/2019-Mar-22_cbttc-hakon/pt_ids.txt', 'w')
out.write('\n'.join(pt_ids))
out.close()



Processing first batch of 100 https://kf-api-dataservice.kidsfirstdrc.org//participants?study_id=SD_JWS3V24D&limit=100
Processing next batch of 100 https://kf-api-dataservice.kidsfirstdrc.org//participants?after=1551727692.931717&study_id=SD_JWS3V24D&limit=100
Processing next batch of 100 https://kf-api-dataservice.kidsfirstdrc.org//participants?after=1551727744.578031&study_id=SD_JWS3V24D&limit=100
Processing next batch of 100 https://kf-api-dataservice.kidsfirstdrc.org//participants?after=1551727796.450033&study_id=SD_JWS3V24D&limit=100
Processing next batch of 100 https://kf-api-dataservice.kidsfirstdrc.org//participants?after=1551727848.798689&study_id=SD_JWS3V24D&limit=100
Processing next batch of 100 https://kf-api-dataservice.kidsfirstdrc.org//participants?after=1551727898.96681&study_id=SD_JWS3V24D&limit=100
Processing next batch of 100 https://kf-api-dataservice.kidsfirstdrc.org//participants?after=1551727948.013785&study_id=SD_JWS3V24D&limit=100
Processing next batch of 100 h

### Get sequencing center by BS ID

In [5]:
def get_seq_center(bs_id):
    p = 5
    bs_id = bs_id.rstrip('\n')
    bs_url = url + "/biospecimens/" + bs_id
    # sys.stderr.write(bs_url + '\n')
    bs_info = ''
    try:
        bs_info = request_flags('GET', bs_url)
    except Exception as e:
        sys.stderr.write('Got error ' + str(e) + ', trying again after a pause of ' + str(p) + ' seconds\n')
        sleep(5)
        try:
            bs_info = request_flags('GET', bs_url)
        except Exception as e1:
            sys.stderr.write('Repeat failed with error ' + str(e1) + '\n')
            exit(1)
    seq_link = bs_info.json()['_links']['sequencing_center']
    seq_info = ''
    try:
        seq_info = request_flags('GET', url + "/" + seq_link)
    except Exception as e:
        sys.stderr.write('Got error ' + str(e) + ', trying again after a pause of ' + str(p) + ' seconds\n')
        sleep(5)
        try:
            seq_info = request_flags('GET', url + "/" + seq_link)
        except Exception as e1:
            sys.stderr.write('Repeat failed with error ' + str(e1) + '\n')
            exit(1)
    sq_id = seq_info.json()['results']['kf_id']
    sq_name = seq_info.json()['results']['name']
    info_list.append("\t".join([bs_id, sq_id, sq_name]))
    

In [6]:
import sys
from requests import request
import concurrent.futures
from time import sleep

ssl_flag = 1
bs_id_file = open('/Users/brownm28/Documents/PORTAL_LOADS/CBTTC_COMPLETE/2020-Jan-21_KF_UPDATE/rna_bs_ids.txt')
bs_id_list = []
url = 'https://kf-api-dataservice.kidsfirstdrc.org'
for line in bs_id_file:
    bs_id_list.append(line.rstrip('\n'))
info_list = []
x = 1
m = 100

with concurrent.futures.ThreadPoolExecutor(40) as sq_exec:
    sq_results = {sq_exec.submit(get_seq_center, bs_id): bs_id for bs_id in bs_id_list}
    for sq_results in concurrent.futures.as_completed(sq_results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' bs ids\n')
        x += 1

sq_file = open('/Users/brownm28/Documents/PORTAL_LOADS/CBTTC_COMPLETE/2020-Jan-21_KF_UPDATE/seq_center_bd_ids.txt', 'w')
sq_file.write('BS_ID\tSQ_ID\tSQ_Value\n')
for line in info_list:
    sq_file.write(line + '\n')
sq_file.close()


Processed 100 bs ids
Processed 200 bs ids
Processed 300 bs ids
Processed 400 bs ids
Processed 500 bs ids
Processed 600 bs ids
Processed 700 bs ids
Processed 800 bs ids
Processed 900 bs ids
Processed 1000 bs ids
